In [ ]:
from IPython.display import display,HTML
def dhtml(str):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family=Akronim&effect=3d';      
    </style><h1 class='font-effect-3d' 
    style='font-family:Akronim; color:#33ffcc;'>
    %s</h1>"""%str))

Reading classics [Deep Learning Models](https://nbviewer.jupyter.org/github/rasbt/deeplearning-models/blob/master/pytorch_ipynb/rnn/rnn_lstm_packed_imdb.ipynb)

In [ ]:
dhtml('Code Modules, Functions, & Classes')

In [ ]:
import numpy as np,pylab as pl
import torch,random,spacy,pandas as pd
import torch.nn as tnn
import torch.nn.functional as tnnf
from torchtext import data as ttdata
from torchtext import datasets as ttds
torch.backends.cudnn.deterministic=True
dev=torch.device('cuda' \
if torch.cuda.is_available() else 'cpu')
nlp=spacy.load('en')

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,\
MaxPooling1D,Dense,LSTM,Embedding
from tensorflow.keras.preprocessing import \
sequence as ksequence
from tensorflow.keras.datasets import imdb
from tensorflow.keras.callbacks import \
ModelCheckpoint,ReduceLROnPlateau

In [ ]:
def bin_accuracy(model,data_loader):
    model.eval()
    correct_pred,num_examples=0,0
    with torch.no_grad():
        for batch_ids,batch_data in enumerate(data_loader):
            text,text_lengths=batch_data.text
            logits=model(text,text_lengths)
            predicted_labels=(torch.sigmoid(logits)>.5).long()
            num_examples+=batch_data.label.size(0)
            correct_pred+=(predicted_labels==\
                           batch_data.label.long()).sum()
        return correct_pred.float()/num_examples*100
def predict_sentiment(model,sentence):
    model.eval()
    tokenized=[tok.text for tok in nlp.tokenizer(sentence)]
    indexed=[ttext.vocab.stoi[t] for t in tokenized]
    length_tensor=torch.LongTensor([len(indexed)])
    tensor=torch.LongTensor(indexed).to(dev)
    tensor=tensor.unsqueeze(1)
    prediction=torch.sigmoid(model(tensor,length_tensor))
    return prediction.item() 

In [ ]:
dhtml('Data')

In [ ]:
random_state=random.seed(12)
ids='train valid test'.split()
ttext=ttdata.Field(tokenize='spacy',
                   include_lengths=True)
tlabel=ttdata.LabelField(dtype=torch.float)
train,test=ttds.IMDB.splits(ttext,tlabel)
train,valid=train\
.split(random_state=random_state,split_ratio=.8)

In [ ]:
pd.DataFrame([len(el) for el in [train,valid,test]],
    index=ids,columns=['length']).T

In [ ]:
random_seed=23
torch.manual_seed(random_seed)
vocab_size=20000; batch_size=128
ttext.build_vocab(train,max_size=vocab_size)
tlabel.build_vocab(train)
num_clases=tlabel.vocab
input_dim=len(ttext.vocab); output_dim=1
train_loader,valid_loader,test_loader=\
ttdata.BucketIterator.splits((train,valid,test),\
batch_size=batch_size,sort_within_batch=True,device=dev)
del train,test,valid
dataloaders={ids[0]:train_loader,
             ids[1]:valid_loader,
             ids[2]:test_loader}
for i in range(3):
    print(ids[i])
    for batch in dataloaders[ids[i]]:
        print(f'text matrix: {batch.text[0].size()}')
        print(f'label vector: {batch.label.size()}')
        break

In [ ]:
# 25,000 movies reviews from IMDB, 
# labeled by sentiment (positive/negative)
num_words,max_length,embedding_vector_len=\
10000,1000,32
(x_train,y_train),(x_test,y_test)=\
imdb.load_data(path="imdb_full.pkl",num_words=num_words,
               skip_top=0,maxlen=max_length,seed=113,
               start_char=1,oov_char=2,index_from=3)
x_test,x_valid,y_test,y_valid=\
train_test_split(x_test,y_test,
                 test_size=.5,random_state=1)

In [ ]:
print(x_train.shape,x_valid.shape,x_test.shape)
print('Label: ',y_train[1]) 
print('Sequence of word indices: \n',x_train[1])
px_train=ksequence\
.pad_sequences(x_train,maxlen=max_length)
px_valid=ksequence\
.pad_sequences(x_valid,maxlen=max_length)
px_test=ksequence\
.pad_sequences(x_test,maxlen=max_length)

In [ ]:
dhtml('Basic RNNs')

In [ ]:
class RNN(tnn.Module):
    def __init__(self,input_dim,embedding_dim,
                 hidden_dim,output_dim):      
        super().__init__()    
        self.embedding=tnn.Embedding(input_dim,embedding_dim)
        self.rnn=tnn.LSTM(embedding_dim,hidden_dim)
        self.fc=tnn.Linear(hidden_dim,output_dim)    
    def forward(self,text,text_length):
        embedded=self.embedding(text)
        packed=tnn.utils.rnn\
        .pack_padded_sequence(embedded,text_length)
        packed_output,(hidden,cell)=self.rnn(packed)
        return self.fc(hidden.squeeze(0)).view(-1)

In [ ]:
def kmodel():
    model=Sequential()
    model.add(Embedding(num_words,embedding_vector_len,
                        input_length=max_length))
    model.add(Conv1D(filters=32,kernel_size=3,
                     padding='same',activation='relu'))
    model.add(MaxPooling1D(pool_size=2))   
    model.add(LSTM(32))    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='nadam',metrics=['accuracy'])    
    return model

In [ ]:
dhtml('Training')

In [ ]:
random_seed=34; learning_rate=1e-4
torch.manual_seed(random_seed)
embedding_dim=128; hidden_dim=256
model=RNN(input_dim,embedding_dim,
          hidden_dim,output_dim)
model=model.to(dev)
optimizer=torch.optim\
.Adam(model.parameters(),lr=learning_rate)

In [ ]:
epochs=10
for epoch in range(epochs):
    model.train()
    for batch_ids,batch_data in enumerate(train_loader):
        text,text_lengths=batch_data.text
        logits=model(text,text_lengths)
        cost=tnnf.binary_cross_entropy_with_logits(logits,
                                                   batch_data.label)
        optimizer.zero_grad()
        cost.backward(); optimizer.step()
        if not batch_ids%100:
            print (f'Epoch: {epoch+1:03d}/{epochs:03d} | '
                   f'Batch: {batch_ids:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.6f}')
    with torch.set_grad_enabled(False):
        print(f'training acc: '
              f'{bin_accuracy(model,train_loader):.2f}%'
              f'\nvalid acc: '
              f'{bin_accuracy(model,valid_loader):.2f}%')
print(f'test acc: {bin_accuracy(model,test_loader):.2f}%')

In [ ]:
kmodel=kmodel()
fw='weights.best.hdf5'
checkpointer=\
ModelCheckpoint(filepath=fw,verbose=2,
                save_best_only=True)
lr_reduction=\
ReduceLROnPlateau(monitor='val_loss',patience=10,
                  verbose=2,factor=.5)
history=kmodel.fit(px_train,y_train,epochs=5,batch_size=128,
                   validation_data=(px_valid,y_valid),
                   callbacks=[checkpointer,lr_reduction])
kmodel.load_weights(fw)
kmodel.evaluate(px_test,y_test)

In [ ]:
dhtml('Predictions')

In [ ]:
print('probability of being positive:')
sent='I really love this movie. The actor team here is so great!'
predict_sentiment(model,sent)

In [ ]:
word2index=imdb.get_word_index()
sent=sent.lower().replace('.','').replace('!','')
sent_index=[]
for word in sent.split():
     sent_index.append(word2index[word])
sent_index=ksequence.pad_sequences([sent_index],
                                   maxlen=max_length)
kmodel.predict(sent_index)